# Google directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/ES/code'
path = '/content/drive/My Drive/ES/code'

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1pAcAZAmcFq145J_jofBoHW6nY7jft6Tp/ES/code


# Libraries

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
from traces_bin_reader.TracesBin import TracesBin
import numpy as np


In [ ]:
from tqdm import tqdm
import pandas as pd
from numpy import unravel_index

In [ ]:
!pip install --upgrade numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 5.0 MB/s 
     |████████████████████████████████| 34.5 MB 1.2 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [ ]:
!pip install --upgrade tbb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 5.3 MB/s 


loading the traces from binary file

In [ ]:
#f= TracesBin("../../traces_CW305/Artix7-100t/0_95V/50MHz/2021-09-21/2021-09-21_50MHz_125MSa_12bit_k0-00_1k.dat")
f = TracesBin("2021-10-25_100t_duDFS_VCC-1.00_freq-50.000MHz_125MSa_12bit_k0-01_1k.dat")

# Data Visualization

In [ ]:
assert f.getNTraces() == 500
assert f.getNSamples() == 134016

In [ ]:
data, plain = f.getAllTraces()

In [ ]:
print(data)
print(data.shape)

[[ -921.   -65. -1680. ...  -501.   355.  1372.]
 [-1228.  -582. -2083. ...  -307.   468.  1275.]
 [-1438.  -582. -2229. ...  -420.   581.  1356.]
 ...
 [-1179.  -953. -2083. ...  -210.   516.  1097.]
 [ -953.  -792. -2309. ...     0.   904.   968.]
 [-1357. -1131. -2229. ...   -81.   888.  1210.]]
(500, 134016)


In [ ]:
print(plain.shape)

(500, 16)


In [ ]:
plain[2][7]

70

In [ ]:
data[1][0]

-1228.0

# CPA
To perform a correlated power analysis attack we will proceed with the following steps:

1. Build a matrix of Estimated Power Model. 
  -> each value of estimated matrix will correspond to hamming distance value between the plain text and possible key value

2. Calculate the Pearson correlation coefficient between the modeled and actual power consumption. Do this for every data point in the traces.

3. Save the results in 16 CSV files, each of them according to 1 byte of the secret key.

4. Observe which subkey guess correlates best to the measured traces.



First we are creating an array with possible key values between 0 and 256 that repressent our guess

In [ ]:
guess = np.arange(0,256)

In [ ]:
guess

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

Define the function to calculating hamming distance between two values

In [ ]:
def hamming_distance(value1,value2):
  n = np.bitwise_xor(value1,value2)
  c = 0
  while n:
    c += 1
    n &= n - 1
  return c

In [ ]:
#check the function 
x = hamming_distance(48,35)
print(x)

3


Define power model matrix with size 500, 256 that will later be populated with intermediate values that are gonna be our power estimate

In [ ]:
power_model = np.zeros((500,256))

CPU example:
For the first byte:
  calculate estimated power values for the first byte of 16 of the encryption key

In [ ]:
for i in range(0,256):
  for t in range(0,500):
    power_model[t][i] = hamming_distance(plain[t][0],guess[i])

run for all 16 bytes

In [ ]:
#initialize a matrix that will correspond to Estimated Power Model
all_power_models = np.zeros((16,500,256))

In [ ]:
for byte in range(0,16):
  for i in range(0,256):
    for t in range(0,500):
      power_model[t][i] = hamming_distance(plain[t][byte],guess[i])
  all_power_models[byte]= power_model

array for correlation values

In [ ]:
#initialize with 0 the array where the correlation coeficients will be stored
corr = np.zeros((134016,256))

Save Real Power Model values inside an np array

In [ ]:
data_arr = np.array(data)
print(data_arr)

[[ -921.   -65. -1680. ...  -501.   355.  1372.]
 [-1228.  -582. -2083. ...  -307.   468.  1275.]
 [-1438.  -582. -2229. ...  -420.   581.  1356.]
 ...
 [-1179.  -953. -2083. ...  -210.   516.  1097.]
 [ -953.  -792. -2309. ...     0.   904.   968.]
 [-1357. -1131. -2229. ...   -81.   888.  1210.]]


In [ ]:
print(power_model.shape)
print(power_model)

(500, 256)
[[1. 2. 2. ... 6. 6. 7.]
 [2. 3. 3. ... 5. 5. 6.]
 [1. 2. 2. ... 6. 6. 7.]
 ...
 [6. 5. 5. ... 3. 3. 2.]
 [5. 4. 6. ... 2. 4. 3.]
 [4. 3. 3. ... 5. 5. 4.]]


# CPU execution

In [ ]:
corr_value = np.corrcoef(power_model[:,2], data_arr[:,2])

In [ ]:
corr_value[0,1]

-0.18816627018770002

CPU: for numpy one iteration takes 15.6 seconds

In [ ]:
for i in tqdm(range(0,256)):
  for t in range(0,134016):
    corr[t][i] = np.corrcoef(power_model[:,i], data_arr[:,t])[0,1]


  0%|          | 0/256 [00:12<?, ?it/s]


KeyboardInterrupt: ignored

# Numba implementation

using numba 1 iteration takes 2 seconds

In [ ]:
from numba import jit, cuda, njit, prange

@njit
def pearson_cor(e1, e2):
    return np.corrcoef(e1, e2)[0,1]
  

In [ ]:
for i in tqdm(range(0,256)):
  for t in range(0,134016):
    
    corr[t][i] = pearson_cor(power_model[:,i], data_arr[:,t])

100%|██████████| 256/256 [06:21<00:00,  1.49s/it]


creating and pandas dataframe and printing the values into a csv file

In [ ]:
df = pd.DataFrame(corr)
df.to_csv("0_with_numba.csv",index=False)

to run for all bytes

In [ ]:
for byte in tqdm(range(0,16)):
  for i in tqdm(range(0,256)):
    for t in range(0,134016):
      corr[t][i] = pearson_cor(all_power_models[byte,:,i], data_arr[:,t])
  df =pd.DataFrame(corr)
  filename = str(byte) + ".csv"
  df.to_csv(filename,index=False)

# Numba parallelization of iteration

In [ ]:
@njit(parallel=True)
def parallel_correlation(power_model,data_arr,corr,byte):
  for i in prange(0,256):
    for t in prange(0,134016):
      corr[t][i] = pearson_cor(all_power_models[byte,:,i], data_arr[:,t])

run for first byte

In [ ]:
parallel_correlation(power_model,data_arr,corr,0)

same operation for all bytes

In [ ]:
for byte in range(0,16):
  parallel_correlation(power_model,data_arr,corr,byte)
  df = pd.DataFrame(corr)
  filename = str(byte) + ".csv"
  df.to_csv(filename,index=False)
  print("file done", byte)

more parallel version

In [ ]:
@njit(parallel=True)
def parallel_correlation_all(power_model,data_arr,corr):
  for byte in prange(0,16):
    for i in prange(0,256):
      for t in prange(0,134016):
        corr[t][i] = pearson_cor(all_power_models[byte,:,i], data_arr[:,t])
    df = pd.DataFrame(corr)
    filename = str(byte) + ".csv"
    df.to_csv(filename,index=False)
    print("file done ",byte)

In [ ]:
parallel_correlation_all(power_model, data_arr, corr)

In [ ]:
guessed_value = np.zeros((16))
max_values = np.zeros((16))

Finding the largest correlation values for each byte

In [ ]:
for byte in range(0,16):
    filename = "results/" + str(byte) + ".csv"
    my_file = pd.read_csv(filename)
    my_array = my_file.to_numpy()
    guessed_value[byte] = unravel_index(my_array.argmax(), my_array.shape)[1]
    max_values[byte] = np.amax(my_array)
    print(byte, " ",guessed_value[byte])

0   255.0
1   193.0
2   170.0
3   204.0
4   215.0
5   81.0
6   61.0
7   253.0
8   16.0
9   10.0
10   232.0
11   83.0
12   151.0
13   252.0
14   195.0
15   197.0
